In [84]:
import pandas as pd
import numpy as np
import os
import pickle as pkl
import h5py
from tqdm import tqdm
import sys

In [3]:
data_dir = "./data/clusterdata-2011-2/"

task_events_header = ['timestamp','missing_info', 'job_id','task_index','machine_id','event_type',
                      'user_name','sched_class','priority','cpu_req','ram_req','space_req','diff_machine']

machine_events_header = ['timestamp','machine_id','event_type','platform_id','cpu_cap','mem_cap']

task_usage_header = ['start_time','end_time','job_id','task_index','machine_id', 'mean_cpu_usage',
                     'canon_memory_usage', 'assign_memory_usage', 'unmapped_cache', 'total_cache', 
                     'max_mem', 'mean_io', 'mean_space', 'max_cpu', 'max_io', 'cpi', 'mai', 'sample',
                     'agg_type', 'sample_cpu_usage']

In [3]:
df = pd.read_csv(data_dir+"task_usage/part-00000-of-00500.csv.gz",header=None)
df.columns = task_usage_header

# Machines that are removed from the cluster were loaded more than X times their mean resource usage.

### Machines that crashed

In [20]:
df = pd.read_csv(data_dir + "machine_events/part-00000-of-00001.csv.gz",header=None)
df.columns = machine_events_header
crash_machines = df[df['event_type'] == 1]['machine_id']


crash_machines = np.unique(crash_machines)
hf = h5py.File('./data/machines_that_crash.h5','w')
hf.create_dataset('ids', data= crash_machines.astype(np.int64))
hf.close()

In [1]:
first_stamp = pd.read_csv(data_dir+"task_usage/part-00000-of-00500.csv.gz",header=None)
mid_stamp = pd.read_csv(data_dir+"task_usage/part-00050-of-00500.csv.gz",header=None)
last_stamp = pd.read_csv(data_dir+"task_usage/part-00100-of-00500.csv.gz",header=None)
first_stamp.columns = task_usage_header
mid_stamp.columns = task_usage_header
last_stamp.columns = task_usage_header
start_time = int(first_stamp.iloc[0]['start_time']/1000000)
mid_time = int(mid_stamp.iloc[-1]['start_time']/1000000)
end_time = int(last_stamp.iloc[-1]['start_time']/1000000)

del first_stamp
del mid_stamp
del last_stamp

NameError: name 'pd' is not defined

### Get Machine IDs of all machines in the first 100 task usage files

In [22]:
hf = h5py.File('./data/machines_that_crash.h5','r')
crash_machines = np.array(hf['ids'])
hf.close()

hf = h5py.File('./data/time_stamps.h5','r')
ts = np.array(hf.get('time'))
hf.close()

# start_time = ts[0]
# end_time = ts[1]

# cols = range(start_time,end_time)

import gc
gc.collect()

1089

In [98]:

all_machines = np.array([])
for i in xrange(101):
    print i
    df = pd.read_csv(data_dir + "task_usage/part-%05d-of-00500.csv.gz" % i,header=None)
    df.columns = task_usage_header
    m = df.groupby('machine_id').size().reset_index(name='counts')
    m_count = m[m['counts'] > 100]['machine_id']
    all_machines = np.append(all_machines, m_count.values)

all_machines = np.unique(all_machines)
hf = h5py.File('./data/machines_in_100.h5','w')
hf.create_dataset('ids', data= all_machines.astype(np.int64))
hf.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [23]:
machine_usage = pd.DataFrame(index=np.array(crash_machines), columns=cols,dtype=np.float16)

In [25]:
for i in tqdm(range(50)):
    df = pd.read_csv(data_dir + "task_usage/part-%05d-of-00500.csv.gz" % i,header=None)
    df.columns = task_usage_header
    df = df[['start_time','machine_id','mean_cpu_usage']]
    df['start_time'] = df['start_time']/1000000
    df['start_time'].astype(np.int)
    df = df[df['machine_id'].isin(crash_machines)].groupby(['start_time', 'machine_id']).sum()
    df.reset_index(level=['machine_id'],inplace=True)
    start_times = df.index.unique()
    for j in start_times:
        dummy = df.loc[j,['machine_id','mean_cpu_usage']].as_matrix().T
        machine_usage[j][dummy[0]] = dummy[1]
    del df
    gc.collect()
    


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [18:58<00:00, 21.51s/it]


In [8]:
df = pd.read_csv(data_dir + "machine_events/part-00000-of-00001.csv.gz",header=None)
df.columns = machine_events_header

In [12]:
df[df['event_type'] == 1]

,timestamp,machine_id,event_type,platform_id,cpu_cap,mem_cap
12477,779231019,5782512,1,HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=,0.5,0.2493
12479,1460219913,4820073668,1,GtXakjpd0CD41brK7k/27s3Eby3RpJKy7taB9S8UQRA=,1.0,1.0000
12480,2071421510,1272981,1,HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=,0.5,0.4995
12481,2580105468,294985247,1,HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=,0.5,0.2493
12483,3016500490,8631300,1,HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=,0.5,0.4995
12496,4018106649,317808289,1,HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=,0.5,0.2493
12498,4617561765,705636,1,HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=,0.5,0.4995
12504,5788416835,3314598368,1,HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=,0.5,0.4995
12505,5991643758,39146853,1,HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=,0.5,0.2493
12506,6384948492,711934,1,HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=,0.5,0.4995


In [75]:
# sdf = pd.SparseDataFrame()
cols = ['total_usage','measure_count']
mean_machine_usage = pd.DataFrame(index=crash_machines,columns=cols)
mean_machine_usage.fillna(0,inplace=True)

In [94]:
for i in tqdm(range(500)):
    df = pd.read_csv(data_dir + "task_usage/part-%05d-of-00500.csv.gz" % i,header=None)
    df.columns = task_usage_header
    df = df[['start_time','machine_id','mean_cpu_usage']]
    df = df[df['machine_id'].isin(crash_machines)].groupby(['machine_id', 'start_time']).agg('sum')
    df.reset_index(level='start_time',inplace=True)
    df = df.groupby('machine_id').agg(['sum','count'])['mean_cpu_usage']
    
    mean_machine_usage['total_usage'] += df['sum']
    mean_machine_usage['measure_count'] += df['count']
#     del df
#     gc.collect()

In [34]:
df.columns = task_usage_header
df = df[['start_time','machine_id','mean_cpu_usage']]

In [92]:
count = 0
for i in range(500):
    
print(df.shape)
df = pd.concat([df,nxt])
print(df.shape)
sys.getsizeof(df) / 1024 / 1024 / 1024

(2520773, 20)
(4873220, 20)


0.7624747082591057

In [159]:
crash_usage = pd.read_csv(data_dir + "machine_events/part-00000-of-00001.csv.gz",header=None)
crash_usage.columns = machine_events_header
crash_usage['mean_cpu_usage'] = 0
crash_usage = crash_usage[crash_usage['event_type'] ==1]
# df = pd.read_csv(data_dir + "task_usage/part-%05d-of-00500.csv.gz" % i,header=None)
# df.columns = task_usage_header

In [136]:
# 10623000000
# crash_usage.head(30)

In [160]:
f = 0
data = pd.read_csv(data_dir + "task_usage/part-%05d-of-00500.csv.gz" % f,header=None)
data.columns = task_usage_header
f+=1
for ind in tqdm(crash_usage.index):
    row = crash_usage.loc[ind]
    while f < 500 and data[(data['end_time'] >= row['timestamp']) & (data['start_time'] <= row['timestamp'])].empty:
        
        data = pd.read_csv(data_dir + "task_usage/part-%05d-of-00500.csv.gz" % f,header=None)
        data.columns = task_usage_header
        f+=1
    data
    temp = data[(data['end_time'] >= row['timestamp']) & (data['start_time'] <= row['timestamp'])]
    crash_usage.loc[ind,'mean_cpu_usage'] = temp[temp['machine_id'] == row['machine_id']].sum()['mean_cpu_usage']
        
#     df[df['end_time'] >= row['timestamp'] & df['start_time'] <= row['timestamp']]
# df[df['end_time'] > crash_usage]
# df.tail()

100%|████████████████████████████████████████████████████████████████████████████| 8957/8957 [1:14:30<00:00,  2.00it/s]


In [144]:
f = 0
data = pd.read_csv(data_dir + "task_usage/part-%05d-of-00500.csv.gz" % f,header=None)
data.columns = task_usage_header

In [181]:
t = data[(data['end_time'] >= 779231019) & (data['start_time'] <= 779231019)]
# t[t['job_id'] == 6251674199]
t

,start_time,end_time,job_id,task_index,machine_id,mean_cpu_usage,canon_memory_usage,assign_memory_usage,unmapped_cache,total_cache,max_mem,mean_io,mean_space,max_cpu,max_io,cpi,mai,sample,agg_type,sample_cpu_usage
0,600000000,900000000,3418309,0,4155527081,0.001562,0.06787,0.07568,0.001156,0.001503,0.06787,2.861000e-06,0.000187,0.039670,0.000357,2.445,0.007243,0,1,0.0
1,600000000,900000000,3418309,1,329150663,0.001568,0.06787,0.07556,0.000319,0.000700,0.06787,5.722000e-06,0.000188,0.033020,0.000929,2.100,0.005791,0,1,0.0
2,600000000,900000000,3418314,0,3938719206,0.000307,0.08044,0.09521,0.000282,0.000670,0.08044,4.768000e-06,0.000184,0.023770,0.000786,5.588,0.020800,0,1,0.0
3,600000000,900000000,3418314,1,351618647,0.000300,0.08044,0.09521,0.000537,0.000870,0.08044,9.537000e-06,0.000183,0.007919,0.002285,5.198,0.020380,0,1,0.0
4,600000000,900000000,3418319,0,431052910,0.000461,0.07715,0.08740,0.000625,0.000852,0.07715,1.907000e-06,0.000204,0.005112,0.000215,2.937,0.009449,0,1,0.0
5,600000000,900000000,3418319,1,257348783,0.000519,0.07678,0.08740,0.000338,0.000721,0.07678,6.676000e-06,0.000204,0.015080,0.000929,3.055,0.010070,0,1,0.0
6,600000000,900000000,3418324,0,5655258253,0.000381,0.08118,0.08936,0.000407,0.000732,0.08118,5.722000e-06,0.000197,0.016240,0.000643,3.880,0.013760,0,1,0.0
7,600000000,900000000,3418324,1,3550322224,0.000327,0.08057,0.08936,0.000293,0.000591,0.08069,1.907000e-06,0.000197,0.015990,0.000072,4.088,0.015690,0,1,0.0
8,600000000,900000000,3418329,0,1303745,0.002518,0.09106,0.10110,0.000448,0.000779,0.09106,2.861000e-06,0.000358,0.021210,0.000215,2.074,0.005869,0,1,0.0
9,600000000,900000000,3418329,1,3894543095,0.002705,0.09082,0.10010,0.000429,0.000711,0.09094,6.676000e-06,0.000358,0.014890,0.000999,2.215,0.006107,0,1,0.0


In [143]:
crash_usage

,timestamp,machine_id,event_type,platform_id,cpu_cap,mem_cap,mean_cpu_usage
12477,779231019,5782512,1,HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=,0.5,0.2493,0.000000
12479,1460219913,4820073668,1,GtXakjpd0CD41brK7k/27s3Eby3RpJKy7taB9S8UQRA=,1.0,1.0000,0.000000
12480,2071421510,1272981,1,HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=,0.5,0.4995,0.000000
12481,2580105468,294985247,1,HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=,0.5,0.2493,0.000000
12483,3016500490,8631300,1,HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=,0.5,0.4995,0.000000
12496,4018106649,317808289,1,HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=,0.5,0.2493,0.134664
12498,4617561765,705636,1,HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=,0.5,0.4995,0.000000
12504,5788416835,3314598368,1,HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=,0.5,0.4995,0.000000
12505,5991643758,39146853,1,HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=,0.5,0.2493,0.000000
12506,6384948492,711934,1,HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=,0.5,0.4995,0.000000


  7%|█████▎                                                                       | 621/8957 [08:32<1:54:35,  1.21it/s]

In [156]:
crash_usage.to_csv('./data/crash_stats_err.csv')

In [162]:
crash_usage['mean_cpu_usage'].astype('bool').sum()/crash_usage.shape[0]

0.29407167578430277

In [164]:
f = 0
data = pd.read_csv(data_dir + "task_usage/part-%05d-of-00500.csv.gz" % f,header=None)
data.columns = task_usage_header

In [170]:
t = data[(data['end_time'] >= 900000000) & (data['start_time'] <= 900000000)]
t[t['machine_id'] == 4820073668]

,start_time,end_time,job_id,task_index,machine_id,mean_cpu_usage,canon_memory_usage,assign_memory_usage,unmapped_cache,total_cache,max_mem,mean_io,mean_space,max_cpu,max_io,cpi,mai,sample,agg_type,sample_cpu_usage
29699,600000000,900000000,4974637670,72,4820073668,0.025120,0.068970,0.083500,0.007645,0.008408,0.071040,0.000592,0.000668,0.257300,0.032780,NaN,NaN,0,1,0.000000
30030,600000000,900000000,4974638024,188,4820073668,0.024380,0.069340,0.083500,0.006844,0.007538,0.069580,0.000557,0.000693,0.147000,0.026860,NaN,NaN,0,1,0.000000
30148,600000000,900000000,4974639465,5,4820073668,0.032780,0.069950,0.083370,0.008499,0.009216,0.070920,0.000622,0.000683,0.275400,0.031980,NaN,NaN,0,1,0.000000
30414,600000000,900000000,4974639809,39,4820073668,0.029360,0.068850,0.083500,0.007271,0.007996,0.069340,0.000567,0.000667,0.213600,0.027010,NaN,NaN,0,1,0.000000
30665,600000000,900000000,4974641217,29,4820073668,0.002518,0.001112,0.006729,0.003330,0.003357,0.009323,0.000002,0.000064,0.000000,0.000500,NaN,NaN,0,1,0.000000
31460,600000000,900000000,4980808964,3,4820073668,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0,0,0.000000
31461,600000000,900000000,4980808964,3,4820073668,0.000000,0.000974,0.003872,0.001188,0.001436,0.001165,0.000000,0.000038,0.000000,0.000000,1.417,NaN,0,0,0.000000
40173,600000000,900000000,5464330466,81,4820073668,0.038090,0.066890,0.076780,0.003891,0.004539,0.067990,0.000039,0.000384,0.114900,0.002285,3.327,NaN,0,0,0.028350
44772,600000000,900000000,5664371117,72,4820073668,0.000061,0.000113,0.001938,0.000007,0.000114,0.000114,0.000011,0.000000,0.000294,0.002285,24.040,NaN,0,0,0.000017
61897,600000000,900000000,6176858948,2771,4820073668,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000002,0.000000,0.000000,NaN,NaN,0,1,0.000000


In [173]:
df

,start_time,end_time,job_id,task_index,machine_id,mean_cpu_usage,canon_memory_usage,assign_memory_usage,unmapped_cache,total_cache,max_mem,mean_io,mean_space,max_cpu,max_io,cpi,mai,sample,agg_type,sample_cpu_usage
0,1.062300e+10,1.080000e+10,6.210691e+09,11.0,4.820095e+09,0.044130,0.010180,0.020260,0.003166,0.003277,0.010510,9.537000e-07,0.000000e+00,0.458000,0.000072,2.8250,NaN,0.0,0.0,0.194600
1,1.062300e+10,1.080000e+10,6.238340e+09,651.0,2.861136e+08,0.034730,0.013180,0.013550,0.000035,0.000154,0.013370,2.146000e-04,9.537000e-07,0.135300,0.004501,1.8640,0.003326,0.0,0.0,0.030640
2,1.062300e+10,1.076100e+10,6.238340e+09,780.0,3.873818e+07,0.015170,0.007980,0.009659,0.000255,0.000548,0.008408,1.965000e-04,4.768000e-06,0.074950,0.003284,2.0360,0.004418,0.0,0.0,0.008362
3,1.062300e+10,1.080000e+10,6.238340e+09,2608.0,3.173374e+08,0.018770,0.013490,0.014720,0.000175,0.000405,0.014130,4.530000e-04,1.907000e-06,0.098140,0.021510,1.1070,0.001259,0.0,0.0,0.008698
4,1.062300e+10,1.071500e+10,6.238340e+09,2627.0,4.302676e+09,0.012500,0.009491,0.010090,0.000079,0.000328,0.009659,2.365000e-04,5.722000e-06,0.069340,0.003216,1.4780,0.002897,0.0,0.0,0.008575
5,1.062300e+10,1.080000e+10,6.251278e+09,737.0,4.182979e+09,0.001352,0.007462,0.007736,0.000082,0.000237,0.007469,2.861000e-06,2.861000e-06,0.005737,0.000428,7.4620,0.026600,0.0,0.0,0.000916
6,1.062300e+10,1.080000e+10,6.251278e+09,845.0,1.302394e+06,0.000551,0.006752,0.007736,0.000074,0.000254,0.006760,2.861000e-06,2.861000e-06,0.003723,0.000357,8.0800,0.031470,0.0,0.0,0.000463
7,1.062300e+10,1.080000e+10,6.251278e+09,1261.0,2.575030e+08,0.000701,0.007355,0.011440,0.000359,0.000500,0.007355,2.861000e-06,4.768000e-06,0.004395,0.000143,4.9060,0.017100,0.0,0.0,0.000843
8,1.062300e+10,1.080000e+10,6.251996e+09,0.0,3.505881e+08,0.032230,0.006256,0.008545,0.001646,0.001720,0.006424,3.281000e-04,1.526000e-05,0.055360,0.006287,2.1030,0.004114,0.0,0.0,0.023280
9,1.062300e+10,1.067000e+10,6.251996e+09,879.0,6.640637e+06,0.046080,0.006134,0.008636,0.001781,0.001835,0.006248,4.740000e-04,1.621000e-05,0.063720,0.004074,1.2830,0.001340,0.0,0.0,0.051510
